<a href="https://colab.research.google.com/github/sahilgupta001/TextSummarizer/blob/master/text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***We are using the BERT(Binary Encoder Representation of Transformers) for training a model. We will use the pre-trained text to text h5 model provided by the hugging face library. The model is trained on a large corpus of text and thius helps learn the word embeddings easily. We have choosen this model to get rid of the large and the long process of model training. Transfer learning is the new state-of-the-art technology and hence we are using a pre-trained model for that.***

# **PART - 1**

***In this part we are going to load the pretrained model from the transformers library and do the required text preprocessing. After that we will get the predictions for all the articles provided to us in the test file and store them in submission file***

In [ ]:
%pip install transformers==2.8.0
import torch
import json
import csv
import numpy as np
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

     |████████████████████████████████| 573kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 29kB/s 
     |████████████████████████████████| 1.1MB 31.6MB/s 
     |████████████████████████████████| 890kB 41.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=bfbda6e38c6665fe65cc7c0a91458101ddc669bfa8c526579ca4e330678e27da
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


***Importing the pretrained model from the Hugging Face Transformers Library***

In [ ]:
pretrained_model = T5ForConditionalGeneration.from_pretrained('t5-small')

***Importing and using the built in tokenizer provided by the Hugging Face Library***

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small', do_lower_case = False)
device = torch.device("cpu")

***Importing the dataset that we have***

In [ ]:
data = pd.read_excel("/content/TASK.xlsx", skiprows = 1)
print(data.shape)
print(data.head())

(1000, 2)
   Unnamed: 0                                       Introduction
0         NaN  Acnesol Gel is an antibiotic that fights bacte...
1         NaN  Ambrodil Syrup is used for treating various re...
2         NaN  Augmentin 625 Duo Tablet is a penicillin-type ...
3         NaN  Azithral 500 Tablet is an antibiotic used to t...
4         NaN  Alkasol Oral Solution is a medicine used in th...


***The only relevant column in this dataset is the text that we want to summarize so we are keeping that only in out dataframe***

In [ ]:
data = data["Introduction"]
print(data.head())

0    Acnesol Gel is an antibiotic that fights bacte...
1    Ambrodil Syrup is used for treating various re...
2    Augmentin 625 Duo Tablet is a penicillin-type ...
3    Azithral 500 Tablet is an antibiotic used to t...
4    Alkasol Oral Solution is a medicine used in th...
Name: Introduction, dtype: object


In [ ]:
def preprocess(text):
  text = text.strip().replace("\n", "")
  text = "summarize: " + text
  return text

for i in range(len(data)):
  data[i] =  preprocess(data[i])


summaries = []

***Now its time to tokenize all the text using thee tokenizer provided to us.***

In [ ]:
def process(text):
  tokenized_text = tokenizer.encode(text, return_tensors="pt").to(device)
  summary_ids = pretrained_model.generate(tokenized_text,
                             num_beams=4,
                             no_repeat_ngram_size=2,
                             min_length=50,
                             max_length=100,
                             early_stopping=True)
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  summaries.append(output)

In [ ]:
for text in data[:10]:
  process(text)

***Saving the summaries in the submission file***

In [ ]:
data = pd.read_excel("/content/TASK.xlsx", skiprows = 1)
data["Summary"] = summaries
data.to_csv("submission.csv", index = False)

# **PART-2**



---
***Now in this secction we are going to make the same function but this function will take a sentence from the user and the lengths of output required by the model.***

In [ ]:
def processSentence(text, min, max):
  tokenized_text = tokenizer.encode(text, return_tensors="pt").to(device)
  summary_ids = pretrained_model.generate(tokenized_text,
                             num_beams=4,
                             no_repeat_ngram_size=2,
                             min_length=min,
                             max_length=max,
                             early_stopping=True)
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return output

***Now we are required to take the input from the user and ask for the minimum andd the maximum length of the summary from the user and then do thee processing of the input to get the output***

In [ ]:
text = input("Enter the text: ")
min = input("Enter the minimum summary length: ")
max = input("Enter the maximum summary length: ")
text = preprocess(text)
output = processSentence(text, int(min), int(max))
print(output)